<a href="https://colab.research.google.com/github/ranton256/cs583_final_project_classify/blob/main/cs583_final_project_classifier_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a final project for CS 583, Drexel Computer Vision
Richard Anton rna63@drexel.edu
See the project readme for more information at:

https://github.com/ranton256/cs583_final_project_classify/blob/main/README.md



In [1]:
!pip install livelossplot --quiet

In [2]:
import copy
import os
import matplotlib.pyplot as plt
import numpy as np
import time

import torch
import torch.nn as nn
import torch.optim as optim

from torch.optim import lr_scheduler

import torchvision
from torchvision import datasets, models, transforms

In [3]:
!wget https://image-net.org/data/tiny-imagenet-200.zip

--2021-05-30 05:14:05--  https://image-net.org/data/tiny-imagenet-200.zip
Resolving image-net.org (image-net.org)... 171.64.68.16
Connecting to image-net.org (image-net.org)|171.64.68.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  19.2MB/s    in 13s     

2021-05-30 05:14:19 (17.7 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]



In [ ]:
!rm -Rf tiny-imagenet-200/
!unzip tiny-imagenet-200.zip


Archive:  tiny-imagenet-200.zip
   creating: tiny-imagenet-200/
  inflating: tiny-imagenet-200/words.txt  
  inflating: tiny-imagenet-200/wnids.txt  
   creating: tiny-imagenet-200/test/
   creating: tiny-imagenet-200/test/images/
  inflating: tiny-imagenet-200/test/images/test_1860.JPEG  
  inflating: tiny-imagenet-200/test/images/test_613.JPEG  
  inflating: tiny-imagenet-200/test/images/test_6272.JPEG  
  inflating: tiny-imagenet-200/test/images/test_2289.JPEG  
  inflating: tiny-imagenet-200/test/images/test_3408.JPEG  
  inflating: tiny-imagenet-200/test/images/test_1693.JPEG  
  inflating: tiny-imagenet-200/test/images/test_389.JPEG  
  inflating: tiny-imagenet-200/test/images/test_2944.JPEG  
  inflating: tiny-imagenet-200/test/images/test_1275.JPEG  
  inflating: tiny-imagenet-200/test/images/test_8104.JPEG  
  inflating: tiny-imagenet-200/test/images/test_2351.JPEG  
  inflating: tiny-imagenet-200/test/images/test_678.JPEG  
  inflating: tiny-imagenet-200/test/images/test_3076

In [ ]:
!ls tiny-imagenet-200/

In [ ]:
#  careful...
!rm -Rf cs583_final_project_classify
# git[sic] a fresh copy.
!git clone https://github.com/ranton256/cs583_final_project_classify.git


In [ ]:
from cs583_final_project_classify import make_subset

original_data_dir = "tiny-imagenet-200"

# This creates a subset of images that are either feline or canine.
make_subset.setup_training_data(original_data_dir)
make_subset.setup_validation_data(original_data_dir)
make_subset.setup_test_data()

In [ ]:
!ls felines_and_canines/

In [ ]:
from cs583_final_project_classify import training

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        #transforms.RandomRotation(degrees=50),
        #transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'felines_and_canines'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=2)
              for x in ['train', 'val','test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

inputs, classes = inputs.to(device), classes.to(device)

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

training.imshow(out.cpu(), title=[class_names[x] for x in classes])

In [ ]:
pretrained = True

if pretrained:
  # Load the pretrained model from pytorch
  vgg16 = models.vgg16_bn(pretrained=pretrained)

  # freeze all layers
  for param in vgg16.features.parameters():
      param.require_grad = False

  # TODO: try fine tuning the layers instead of freezing.
else:
  # create a fresh model with VGG16 architecture.
  vgg16 = models.vgg16_bn(pretrained=False)


print(vgg16.classifier[6].out_features) # 1000 

# replace classifier layers
num_features = vgg16.classifier[6].in_features
features = list(vgg16.classifier.children())[:-1] # strip last layer
features.extend([nn.Linear(num_features, len(class_names))]) ## new last layer
vgg16.classifier = nn.Sequential(*features) # replace classifier in model

# This is how you load a saved model.
#vgg16.load_state_dict(torch.load("../input/vgg16bn/vgg16_bn.pth"))


print(vgg16)

model_ft = vgg16

criterion = nn.CrossEntropyLoss()

if torch.cuda.is_available():
  model_ft = model_ft.cuda()
  criterion = criterion.cuda()

# Observe that all parameters are being optimized
#optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
#optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9, nesterov=True)


# Trying adam
#optimizer_ft = torch.optim.Adam(model_ft.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
#optimizer_ft = torch.optim.Adam(model_ft.parameters(), lr=0.005, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
#optimizer_ft = torch.optim.Adam(model_ft.parameters())

optimizer_ft = torch.optim.Adam(model_ft.parameters(), lr=0.002, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=True)
#optimizer_ft = torch.optim.Adam(model_ft.parameters(), lr=0.008, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=True)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)







In [ ]:
model_ft = training.train_model(model_ft, dataloaders, dataset_sizes, device, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=50)

In [ ]:
training.visualize_model(model_ft, dataloaders, class_names, device)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os
model_save_name = input("Enter model filename") 
#'pretrained_classifier.pt'
model_save_name = model_save_name + '.pt'
path = os.path.join(F"/content/gdrive/My Drive/cs583_final_project_models/", model_save_name)
torch.save(model_ft.state_dict(), path)

In [ ]:
!ls '/content/gdrive/My Drive/cs583_final_project_models'

## References
* https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
* Very Deep Convolutional Networks for Large-Scale Image Recognition, Karen Simonyan and Andrew Zisserman, 2015, https://arxiv.org/abs/1409.1556
* Tiny imagenet: http://cs231n.stanford.edu/reports/2017/pdfs/935.pdf
* Saving to Google Drive from colab https://medium.com/@ml_kid/how-to-save-our-model-to-google-drive-and-reuse-it-2c1028058cb2
* Keras label names for imagenet: https://github.com/raghakot/keras-vis/blob/master/resources/imagenet_class_index.json


